In [29]:
import hashlib, json, sys

def hashMe(msg=""):
    # For convenience, this is a helper function that wraps our hashing algorithm
    if type(msg)!=str:
        msg = json.dumps(msg,sort_keys=True)  # If we don't sort keys, we can't guarantee repeatability!
        
    if sys.version_info.major == 2:
        return unicode(hashlib.sha256(msg).hexdigest(),'utf-8')
    else:
        return hashlib.sha256(str(msg).encode('utf-8')).hexdigest()

msg = json.dumps(msg,sort_keys=True) -> str이 아닌 msg는 알파벳순으로 정렬하여 str로 다시환산해줌

sys.version_info.major -> 현재 파이썬버전이 2이면 2, 3이면 3을 출력

hashlib.sha256(str(msg).encode('utf-8')).hexdigest() -> msg를 str로 바꾸고utf-8로 인코딩한후 hexdigest를 통해 sha256해싱을 수행

In [30]:
import random
random.seed(0)

def makeTransaction(maxValue=3):
    # This will create valid transactions in the range of (1,maxValue)
    sign      = int(random.getrandbits(1))*2 - 1   # This will randomly choose -1 or 1
    amount    = random.randint(1,maxValue)
    alicePays = sign * amount
    bobPays   = -1 * alicePays
    # By construction, this will always return transactions that respect the conservation of tokens.
    # However, note that we have not done anything to check whether these overdraft an account
    return {u'Alice':alicePays,u'Bob':bobPays}

txnBuffer = [makeTransaction() for i in range(30)]
print(txnBuffer)
len(txnBuffer)

[{'Alice': 2, 'Bob': -2}, {'Alice': 2, 'Bob': -2}, {'Alice': -2, 'Bob': 2}, {'Alice': 3, 'Bob': -3}, {'Alice': -2, 'Bob': 2}, {'Alice': 2, 'Bob': -2}, {'Alice': 2, 'Bob': -2}, {'Alice': -3, 'Bob': 3}, {'Alice': 1, 'Bob': -1}, {'Alice': 1, 'Bob': -1}, {'Alice': -1, 'Bob': 1}, {'Alice': 1, 'Bob': -1}, {'Alice': 2, 'Bob': -2}, {'Alice': 3, 'Bob': -3}, {'Alice': 3, 'Bob': -3}, {'Alice': 3, 'Bob': -3}, {'Alice': 1, 'Bob': -1}, {'Alice': -1, 'Bob': 1}, {'Alice': 1, 'Bob': -1}, {'Alice': 3, 'Bob': -3}, {'Alice': -2, 'Bob': 2}, {'Alice': 1, 'Bob': -1}, {'Alice': -2, 'Bob': 2}, {'Alice': -3, 'Bob': 3}, {'Alice': 1, 'Bob': -1}, {'Alice': 3, 'Bob': -3}, {'Alice': -2, 'Bob': 2}, {'Alice': 3, 'Bob': -3}, {'Alice': -1, 'Bob': 1}, {'Alice': 3, 'Bob': -3}]


30

sign은 -1과 1중 하나로 결정되고, amount는 1에서 3까지의 값을 임의로 발생시킴. 엘리스는 -3~3범위의 값을 지불하고, bob은 그 반대값을 지불.
해당 예제에서는 각 Transaction행위마다 엘리스와 밥에대해 임의의 수만큼 거래를 발생시킨 것.
결과값으로는 dictionary 형식의 데이터 아웃풋을 출력함

txnBuffer는 30회동안 발생한 Tx를 기록한 List가 됨.

In [31]:
def updateState(txn, state):
    # Inputs: txn, state: dictionaries keyed with account names, holding numeric values for transfer amount (txn) or account balance (state)
    # Returns: Updated state, with additional users added to state if necessary
    # NOTE: This does not not validate the transaction- just updates the state!
    
    # If the transaction is valid, then update the state
    state = state.copy() # As dictionaries are mutable, let's avoid any confusion by creating a working copy of the data.
    for key in txn:
        if key in state.keys():
            state[key] += txn[key]
        else:
            state[key] = txn[key]
    return state

dictionary 자료형은 key와 value로 구성되어있다. 즉 (Key, value)의 형태로 저장되고있음.

In [32]:
def isValidTxn(txn,state):
    # Assume that the transaction is a dictionary keyed by account names

    # Check that the sum of the deposits and withdrawals is 0
    if sum(txn.values()) is not 0:
        return False
    
    # Check that the transaction does not cause an overdraft
    for key in txn.keys():
        if key in state.keys(): 
            acctBalance = state[key]
        else:
            acctBalance = 0
        if (acctBalance + txn[key]) < 0:
            return False
    
    return True

txn는 거래내역, state는 현재 잔고 내역임
txn에는 누가 얼마를 누구에게 전달하는지가 포함되어있으므로, 총 합이 0이 아니라면 거짓임
위의 isValidTxn은 단지 해당 Tx가 올바르게 발생된 거래인지를 나타내는 용도.
overdraft는 잔고보다 많은 돈을 송금하는경우를 의미.. for문 이하의 의미는 다음과 같다.

for key in txn.keys(): -> txn의 key값들에서, (즉 거래에 참여하는 사람들, 엘리스와 밥)

if key in state.keys():  -> 만약 state의 key값에 해당하는 key값이 있다면,,,,, (즉 거래내역을 발생한 유저들이 네트워크 안에 있는 유저라면)

acctBalance = state[key], -> acctBalance에 해당 key의 state를 대입. 즉 txn에 엘리스와 밥이 거래가 이루어졌다면, accbalance에는 엘리스와 밥의 현재 balance가 임시적으로 저장된다.


 else:
            acctBalance = 0   -> 만약 txn의 거래유저가 네트워크에 없다면, acctbalance는 0이됨.
            
 if (acctBalance + txn[key]) < 0:
            return False              -> 만약 acctbalance값(=state[key])과 txn[key]의 합이 0보다 작다면, False를 반환
            
            즉 txn에서는 돈을 빼내게되는데, 현재 state의 잔고보다 많을 돈을 빼내려고하게되면 오류를 출력시키는 것임.

In [33]:
state = {u'Alice':5,u'Bob':5}

print(isValidTxn({u'Alice': -3, u'Bob': 3},state))  # Basic transaction- this works great!
print(isValidTxn({u'Alice': -4, u'Bob': 3},state))  # But we can't create or destroy tokens!
print(isValidTxn({u'Alice': -6, u'Bob': 6},state))  # We also can't overdraft our account.
print(isValidTxn({u'Alice': -4, u'Bob': 2,'Lisa':2},state)) # Creating new users is valid
print(isValidTxn({u'Alice': -4, u'Bob': 3,'Lisa':2},state)) # But the same rules still apply!

True
False
False
True
False


즉 isValidTxn을 통해, 나가는 돈과 들어오는 돈의 합은 같아야하며, overdraft를 유발하는 Tx는 False를 출력하여 사전에 방지할 수 있음

In [34]:
state = {u'Alice':50, u'Bob':50}  # Define the initial state
genesisBlockTxns = [state]
genesisBlockContents = {u'blockNumber':0,u'parentHash':None,u'txnCount':1,u'txns':genesisBlockTxns}
genesisHash = hashMe( genesisBlockContents )
genesisBlock = {u'hash':genesisHash,u'contents':genesisBlockContents}
genesisBlockStr = json.dumps(genesisBlock, sort_keys=True)

L1 - 초기 state로 엘리스, 밥만 존재하는 네트워크를 구성

L2 - 제네시스 블럭에 초기 네트워크 상태를 저장

L3 - 제네시스 블럭 contents에 blocknumber를 기입. parenthash는 이전해시가 없으므로 none. txn 카운트도 1. u'txns에는 genesisblocktxn을 저장

여기서 u'blocknumber, u'parenthash, y'txncount는 블록 헤더에 해당하며, u'txns'이 실제 블록의 콘텐츠가 된다. u'xns'에만 Tx가 기입됨.

L4 - 제네시스해시에서는 초기 Tx(초기 네트워크 자본 상태)를 해싱한 결과를 저장.

L5 - genesisBlock -> 블록의 해싱값과 원본값을 딕셔너리 형태로 저장.

L6 - 제네시스 블록의 내용을 사전순으로 정렬하고 , 데이터 형식을 str로 변경 sort_key=True 는 딕셔너리의 key를 정렬한다는 의미. 
기본값은 False임.



In [35]:
chain = [genesisBlock]

genesisBlock을 chain이라는 변수에 List형식으로 집어넣음.
한편 genesisBlock은 dictionary형식으로, 최초 정보에 대한 해싱값을 'Key'로, 최초 정보 그자체를 'Value'로 갖고있다.

In [36]:
def makeBlock(txns,chain):
    parentBlock = chain[-1]
    parentHash  = parentBlock[u'hash']
    blockNumber = parentBlock[u'contents'][u'blockNumber'] + 1
    txnCount    = len(txns)
    blockContents = {u'blockNumber':blockNumber,u'parentHash':parentHash,
                     u'txnCount':len(txns),'txns':txns}
    blockHash = hashMe( blockContents )
    block = {u'hash':blockHash,u'contents':blockContents}
    
    return block

txn count는 block에 들어가는 tx의 갯수 (txns), lens를 통해 값을 반환

블록콘텐츠에는 값을 넣는데, txns라는 값이 블록을 생성하기 직전까지 모인 Transaction의 덩어리라고 봐도 될듯.




In [37]:
blockSizeLimit = 5  # Arbitrary number of transactions per block- 
               #  this is chosen by the block miner, and can vary between blocks!

while len(txnBuffer) > 0:
    bufferStartSize = len(txnBuffer)
    
    ## Gather a set of valid transactions for inclusion
    txnList = []
    while (len(txnBuffer) > 0) & (len(txnList) < blockSizeLimit):
        newTxn = txnBuffer.pop()
        validTxn = isValidTxn(newTxn,state) # This will return False if txn is invalid
        
        if validTxn:           # If we got a valid state, not 'False'
            txnList.append(newTxn)
            state = updateState(newTxn,state)
        else:
            print("ignored transaction")
            sys.stdout.flush()
            continue  # This was an invalid transaction; ignore it and move on
        
    ## Make a block
    myBlock = makeBlock(txnList,chain)
    chain.append(myBlock)    

txnBuffer>0인 동안 , bufferStartSize-> txnBuffer의 크기로 정한다.

txnList는 처음에 공백리스트로 시작하고 txnBuffer 가 양수이면서 동시에, txnList의 크기가 블록사이즈 최대치인  5보다 작은 동안에 아래의 명령을 수행

A.pop()은 A list의 요소중 맨 마지막요소를 빼내어 출력하는 함수. 실행 후 A는 맨 마지막 List가 삭제되어 있음.

즉 txnBuffer는 지금 노드의 버퍼에 Txn이 차있는데, 그중에서 가장 마지막에 존재한 txn을 꺼내어 newTxn으로 옮기는 과정임.

newTxn은 isValidTxn함수를 통해 검증됨(True,False로 output이 나옴), 해당 결과를 validTxn에 넣음. 

만약 valid라면 txnList에 이어붙이고, 아니라면(validTxn->False라면) 메세지를 출력하고 sys.stdout.flush()를 실행. sys.stdout.flush()는 버퍼 비우는용도

정리 : While문은 tx가 4개가 될때까지 txn버퍼의 tx를 가져와서 txnList에 넣는과정임. 해당 txnList 그리고 chain이 makeBlock의 변수로 들어감. chain은 previous hash를 만들기위해 필요함. 이러한 정보를 토대로 만들어진 블록을 myBlock이라는 변수에 넣고, 이를 chain에 다시 append함.



In [38]:
import json, sys
print(len(json.dumps({"c": 5}, sort_keys=False)))

print(len([5,5]))

sys.stdout.write("hi")

8
2
hi

In [39]:
chain[0]

{'hash': '7c88a4312054f89a2b73b04989cd9b9e1ae437e1048f89fbb4e18a08479de507',
 'contents': {'blockNumber': 0,
  'parentHash': None,
  'txnCount': 1,
  'txns': [{'Alice': 50, 'Bob': 50}]}}

In [40]:
chain[1]

{'hash': '7a91fc8206c5351293fd11200b33b7192e87fad6545504068a51aba868bc6f72',
 'contents': {'blockNumber': 1,
  'parentHash': '7c88a4312054f89a2b73b04989cd9b9e1ae437e1048f89fbb4e18a08479de507',
  'txnCount': 5,
  'txns': [{'Alice': 3, 'Bob': -3},
   {'Alice': -1, 'Bob': 1},
   {'Alice': 3, 'Bob': -3},
   {'Alice': -2, 'Bob': 2},
   {'Alice': 3, 'Bob': -3}]}}

In [49]:
chain[2]

{'hash': '7dfbb47641f8aa4ccf9095647c6190488d63b1f47ad80a0adb86cabc40ab5058',
 'contents': {'blockNumber': 2,
  'parentHash': '7a91fc8206c5351293fd11200b33b7192e87fad6545504068a51aba868bc6f72',
  'txnCount': 5,
  'txns': [{'Alice': 1, 'Bob': -1},
   {'Alice': -3, 'Bob': 3},
   {'Alice': -2, 'Bob': 2},
   {'Alice': 1, 'Bob': -1},
   {'Alice': -2, 'Bob': 2}]}}

In [50]:
chain[3]

{'hash': 'af1de155dee76a6c1e87cbea8657850f0723dbf0d80b51d9430f71d94c4e5eca',
 'contents': {'blockNumber': 3,
  'parentHash': '7dfbb47641f8aa4ccf9095647c6190488d63b1f47ad80a0adb86cabc40ab5058',
  'txnCount': 5,
  'txns': [{'Alice': 3, 'Bob': -3},
   {'Alice': 1, 'Bob': -1},
   {'Alice': -1, 'Bob': 1},
   {'Alice': 1, 'Bob': -1},
   {'Alice': 3, 'Bob': -3}]}}

In [51]:
chain[4]

{'hash': 'fb8c5cedc82b3bc8178e49cab1f86df41cfabcf3c7f92f55a83ecb7bb49a80b5',
 'contents': {'blockNumber': 4,
  'parentHash': 'af1de155dee76a6c1e87cbea8657850f0723dbf0d80b51d9430f71d94c4e5eca',
  'txnCount': 5,
  'txns': [{'Alice': 3, 'Bob': -3},
   {'Alice': 3, 'Bob': -3},
   {'Alice': 2, 'Bob': -2},
   {'Alice': 1, 'Bob': -1},
   {'Alice': -1, 'Bob': 1}]}}

In [52]:
chain[5]

{'hash': 'c38c81cee85616f1cbc105ed4d6e470104590e348898fabf72bb5aab5625e1e8',
 'contents': {'blockNumber': 5,
  'parentHash': 'fb8c5cedc82b3bc8178e49cab1f86df41cfabcf3c7f92f55a83ecb7bb49a80b5',
  'txnCount': 5,
  'txns': [{'Alice': 1, 'Bob': -1},
   {'Alice': 1, 'Bob': -1},
   {'Alice': -3, 'Bob': 3},
   {'Alice': 2, 'Bob': -2},
   {'Alice': 2, 'Bob': -2}]}}

In [53]:
chain[6]

{'hash': '165c6c03b8c3ed8ebf0f4b091b3cf00e40f06b41db3db792d426736007b68872',
 'contents': {'blockNumber': 6,
  'parentHash': 'c38c81cee85616f1cbc105ed4d6e470104590e348898fabf72bb5aab5625e1e8',
  'txnCount': 5,
  'txns': [{'Alice': -2, 'Bob': 2},
   {'Alice': 3, 'Bob': -3},
   {'Alice': -2, 'Bob': 2},
   {'Alice': 2, 'Bob': -2},
   {'Alice': 2, 'Bob': -2}]}}

In [54]:
chain[7]

{'hash': 'e23552dcd00f96b842111af8376eb396cf1419d72583d7b6a60d45fc0026c82f',
 'contents': {'blockNumber': 7,
  'parentHash': '165c6c03b8c3ed8ebf0f4b091b3cf00e40f06b41db3db792d426736007b68872',
  'txnCount': 5,
  'txns': [{'Alice': 1, 'Bob': -1},
   {'Alice': -3, 'Bob': 3},
   {'Alice': 2, 'Bob': -2},
   {'Alice': 3, 'Bob': -3},
   {'Alice': 1, 'Bob': -1}]}}

In [55]:
chain[8]

IndexError: list index out of range

In [41]:
state

{'Alice': 72, 'Bob': 28}

In [42]:
def checkBlockHash(block):
    # Raise an exception if the hash does not match the block contents
    expectedHash = hashMe( block['contents'] )
    if block['hash']!=expectedHash:
        raise Exception('Hash does not match contents of block %s'%
                        block['contents']['blockNumber'])
    return

chenckBlockHash(block) -> 해당 block의 hash 값이, 콘텐츠를 제대로 해싱한것이 맞는지 확인

In [43]:
 def checkBlockValidity(block,parent,state):    
    # We want to check the following conditions:
    # - Each of the transactions are valid updates to the system state
    # - Block hash is valid for the block contents
    # - Block number increments the parent block number by 1
    # - Accurately references the parent block's hash
    parentNumber = parent['contents']['blockNumber']
    parentHash   = parent['hash']
    blockNumber  = block['contents']['blockNumber']
    
    # Check transaction validity; throw an error if an invalid transaction was found.
    for txn in block['contents']['txns']: # block의 컨텐츠항목의 txns를 참조하여, 트랜잭션내 모든 txn에 대해..
        if isValidTxn(txn,state):
            state = updateState(txn,state)   # 유효한 txn이었다면, state를 업데이트
        else:
            raise Exception('Invalid transaction in block %s: %s'%(blockNumber,txn))

    checkBlockHash(block) # Check hash integrity; raises error if inaccurate

    if blockNumber!=(parentNumber+1): #블록넘버 확인
        raise Exception('Hash does not match contents of block %s'%blockNumber)

    if block['contents']['parentHash'] != parentHash: #블록의 컨텐츠항목 내부의 parentHash가 , 실제 parentHash랑 다르다면,,,,
        raise Exception('Parent hash not accurate at block %s'%blockNumber)
    
    return state

블록검증부

1) Hashing 확인 - 해당 블록의 컨텐츠에대한 Hash값이 맞나 확인 + 이전블록의 Hash값 존재하는지 확인

2) blocknumber 확인 + TX가 유효한 TX인지 검증

코드 내부적으로 확인해보면, 블록넘버 해쉬 등등 기타조건이 만족하지않은경우 예외처리가됨

만약 txn이 유효하다면 state는 일단 업데이트 되는듯?

raise 예외처리의경우 만약 에러가 발생하면, raise밑의 코드는 실행되지 않음.

만약 txn은 유효하고 블록의 정보가 틀렸다면,, state는 업데이트되고 아래 if문에서 예외처리가 발생할것임..
예외처리시 블록 생성을 막는 수단이 피룡할듯..


In [44]:
def checkChain(chain):
    # Work through the chain from the genesis block (which gets special treatment), 
    #  checking that all transactions are internally valid,
    #    that the transactions do not cause an overdraft,
    #    and that the blocks are linked by their hashes.
    # This returns the state as a dictionary of accounts and balances,
    #   or returns False if an error was detected

    
    ## Data input processing: Make sure that our chain is a list of dicts
    if type(chain)==str:
        try:
            chain = json.loads(chain)
            assert( type(chain)==list)
        except:  # This is a catch-all, admittedly crude
            return False
    elif type(chain)!=list:
        return False
    
    state = {} # state 초기화
    ## Prime the pump by checking the genesis block
    # We want to check the following conditions:
    # - Each of the transactions are valid updates to the system state
    # - Block hash is valid for the block contents

    for txn in chain[0]['contents']['txns']: # 제네시스블록에 있는 트랜잭션들을 전부다 뜯어봐서
        state = updateState(txn,state) #state를 업데이트하고
    checkBlockHash(chain[0]) #불록해시도 검사하고~
    parent = chain[0] # 제네시스블록에 대한 항을 따로 분리해놓고, 아래에는 나머지 블록에대한검증부분을 다루고잇음.
    
    ## Checking subsequent blocks: These additionally need to check
    #    - the reference to the parent block's hash
    #    - the validity of the block number
    for block in chain[1:]:
        state = checkBlockValidity(block,parent,state)   # 체인에 잇는 블록마다 유효성 검사하여 state를 업데이트하고, 
        parent = block
        
    return state   # state를 업데이트... 즉 checkchain의 결과값은 단지 state임


json.dumps() / json.loads는 각각 데이터를 인코딩/ 디코딩하는 과정임.

인코딩된 chain이 str타입이라면, chain을 다시 디코딩하여, list형식으로 바꿈. 원래 chain은 사전형식데이터의 list였는데, 이것이 str로 변환되어 저장되어있던것임. 이를 해독하기위해선 다시 json.loads를 통해 디코딩하여 일일이 맞춰봐야함.

In [45]:
checkChain(chain)

{'Alice': 72, 'Bob': 28}

In [46]:
chainAsText = json.dumps(chain,sort_keys=True)
checkChain(chainAsText)

{'Alice': 72, 'Bob': 28}

이미 chain은만들어진 상태에서 검증만 하는 작업. output 결과로 state만 출력한다


In [47]:
import copy
nodeBchain = copy.copy(chain)
nodeBtxns  = [makeTransaction() for i in range(5)]
newBlock   = makeBlock(nodeBtxns,nodeBchain)

새로운 블록이 노드에 전달된경우, 검증을 시작하고 검증결과 올바르다면 state를 업데이트함.

nodeBchain은 기존의 chain을 카피한것.

nodeBtxn은 이때 새로 발생한 txn를 지칭

newBlock은 새로발생한 txn를 체인에 연결하는것.



In [56]:
nodeBchain

[{'hash': '7c88a4312054f89a2b73b04989cd9b9e1ae437e1048f89fbb4e18a08479de507',
  'contents': {'blockNumber': 0,
   'parentHash': None,
   'txnCount': 1,
   'txns': [{'Alice': 50, 'Bob': 50}]}},
 {'hash': '7a91fc8206c5351293fd11200b33b7192e87fad6545504068a51aba868bc6f72',
  'contents': {'blockNumber': 1,
   'parentHash': '7c88a4312054f89a2b73b04989cd9b9e1ae437e1048f89fbb4e18a08479de507',
   'txnCount': 5,
   'txns': [{'Alice': 3, 'Bob': -3},
    {'Alice': -1, 'Bob': 1},
    {'Alice': 3, 'Bob': -3},
    {'Alice': -2, 'Bob': 2},
    {'Alice': 3, 'Bob': -3}]}},
 {'hash': '7dfbb47641f8aa4ccf9095647c6190488d63b1f47ad80a0adb86cabc40ab5058',
  'contents': {'blockNumber': 2,
   'parentHash': '7a91fc8206c5351293fd11200b33b7192e87fad6545504068a51aba868bc6f72',
   'txnCount': 5,
   'txns': [{'Alice': 1, 'Bob': -1},
    {'Alice': -3, 'Bob': 3},
    {'Alice': -2, 'Bob': 2},
    {'Alice': 1, 'Bob': -1},
    {'Alice': -2, 'Bob': 2}]}},
 {'hash': 'af1de155dee76a6c1e87cbea8657850f0723dbf0d80b51d9430f71d9

In [59]:
nodeBtxns

[{'Alice': 1, 'Bob': -1},
 {'Alice': -3, 'Bob': 3},
 {'Alice': 2, 'Bob': -2},
 {'Alice': 3, 'Bob': -3},
 {'Alice': 1, 'Bob': -1}]

In [60]:
newBlock

{'hash': 'e23552dcd00f96b842111af8376eb396cf1419d72583d7b6a60d45fc0026c82f',
 'contents': {'blockNumber': 7,
  'parentHash': '165c6c03b8c3ed8ebf0f4b091b3cf00e40f06b41db3db792d426736007b68872',
  'txnCount': 5,
  'txns': [{'Alice': 1, 'Bob': -1},
   {'Alice': -3, 'Bob': 3},
   {'Alice': 2, 'Bob': -2},
   {'Alice': 3, 'Bob': -3},
   {'Alice': 1, 'Bob': -1}]}}

In [48]:
print("Blockchain on Node A is currently %s blocks long"%len(chain))

try:
    print("New Block Received; checking validity...")
    state = checkBlockValidity(newBlock,chain[-1],state) # Update the state- this will throw an error if the block is invalid!
    chain.append(newBlock)
except:
    print("Invalid block; ignoring and waiting for the next block...")

print("Blockchain on Node A is now %s blocks long"%len(chain))

Blockchain on Node A is currently 7 blocks long
New Block Received; checking validity...
Blockchain on Node A is now 8 blocks long


위에서 만든 nodeBtxns, newBlock는 아직 검증이된것이 아니므로 위의 예제를통해 validity 체크를 진행함